In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
sectors = ["Materials", "Industrials","Health Care","Real Estate","Consumer Discretionary","Financials",
                "Utilities","Information Technology","Energy","Consumer Staples","Communication Services"]

ret = pd.read_csv('../data/mid_cap_all_sectors_ret.csv', index_col='date') * 100
ret = ret.stack()
df = ret.reset_index()
df.columns = ['date', 'sector', 'ret']
df = df.set_index(['date', 'sector'])

unique_dates = df.index.get_level_values('date').unique()
n = int(len(unique_dates) * 0.8)
train_n = int(n * 0.8)
tmp_dates = unique_dates[:n]
train_dates = tmp_dates[:train_n]
valid_dates = tmp_dates[train_n:]
test_dates = unique_dates[n:]

train_df = df.loc[train_dates]
valid_df = df.loc[valid_dates]
full_df = df.loc[tmp_dates]
test_df = df.loc[test_dates]

In [ ]:
test_dates

In [25]:
ret = pd.read_csv('../data/mid_cap_all_sectors_ret.csv', index_col='date')

In [26]:
ret.loc[test_dates].to_csv('../data/test_data.csv')

In [29]:
data = pd.read_csv('../data/test_data.csv', index_col='date') * 100
data = data.stack()
df = data.reset_index()
df.columns = ['date', 'sector', 'ret']
df = df.set_index(['date', 'sector'])

In [30]:
df

ret
date       sector                          
2019-09-26 Materials              -0.969016
           Industrials            -0.398503
           Health Care            -2.327705
           Real Estate             0.764402
           Consumer Discretionary -0.718403
...                                     ...
2024-08-30 Utilities               0.892477
           Information Technology  0.424088
           Energy                 -0.136165
           Consumer Staples        0.638347
           Communication Services  0.881521

[13651 rows x 1 columns]

In [31]:
test_df

ret
date       sector                          
2019-09-26 Materials              -0.969016
           Industrials            -0.398503
           Health Care            -2.327705
           Real Estate             0.764402
           Consumer Discretionary -0.718403
...                                     ...
2024-08-30 Utilities               0.892477
           Information Technology  0.424088
           Energy                 -0.136165
           Consumer Staples        0.638347
           Communication Services  0.881521

[13651 rows x 1 columns]

In [32]:
sector_df = test_df.loc[(slice(None), "Materials"), :]

In [33]:
sector_df

,,ret
date,sector,
2019-09-26,Materials,-0.969016
2019-09-27,Materials,-0.335666
2019-09-30,Materials,0.550732
2019-10-01,Materials,-1.704612
2019-10-02,Materials,-1.800228
...,...,...
2024-08-26,Materials,0.253561
2024-08-27,Materials,-0.717639
2024-08-28,Materials,-1.016826


In [2]:
def data_to_tensor(data, dtype=torch.float32):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    return torch.tensor(np.array(data), dtype=dtype).to(device)

class CnnSeqDataset(torch.utils.data.Dataset):
    def __init__(self, data, seq_n, exclude_date_start = '2008-08-01', exclude_date_end ='2009-04-01'):
        df = data.swaplevel().sort_index().copy()
        sample_index = df.groupby('sector').shift(seq_n-1).dropna().index.tolist()
        
        self.data_list = []
        for sample in sample_index:
            if pd.to_datetime(exclude_date_start) <= pd.to_datetime(sample[1]) <= pd.to_datetime(exclude_date_end):
                continue
            else:
                data_tensor = data_to_tensor(df.loc[:sample].iloc[-seq_n:].T)
                data_tuple = (data_tensor, data_tensor)
                self.data_list.append(data_tuple)

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

In [3]:
data = CnnSeqDataset(test_df, 100)

In [11]:
train_df.swaplevel().sort_index().groupby('sector').shift(100-1)

ret
sector                 date                
Communication Services 2000-01-04       NaN
                       2000-01-05       NaN
                       2000-01-06       NaN
                       2000-01-07       NaN
                       2000-01-10       NaN
...                                     ...
Utilities              2018-09-21  0.092864
                       2018-09-24  0.881156
                       2018-09-25  0.975488
                       2018-09-26 -0.682322
                       2018-09-27 -2.045993

[51854 rows x 1 columns]